In [ ]:
import ipywidgets as ipw
from aiida.orm import load_node
from aiidalab_widgets_base import CodQueryWidget, SmilesWidget, StructureExamplesWidget
from aiidalab_widgets_base import StructureBrowserWidget, StructureManagerWidget, StructureUploadWidget
from aiidalab_widgets_base import OptimadeQueryWidget

from wizard import WizardApp
from structures import StructureSelectionStep
from bands import ComputeBandsSubmitWidget
from process import WorkChainSelector

from aiida.orm import StructureData

OptimadeQueryWidget.title = "OPTIMADE"  # monkeypatch


# Create the application steps
structure_manager_widget = StructureManagerWidget(
    importers=[
        StructureUploadWidget(title="From computer"),
        CodQueryWidget(title="COD"),
        StructureBrowserWidget(title="AiiDA database"),
        OptimadeQueryWidget(embedded=True),
        SmilesWidget(title="SMILES"),  # requires OpenBabel! 
        StructureExamplesWidget(
            title="From Examples",
            examples=[
                ("Silicon", "miscellaneous/structures/Si.xyz"),
                ("Silicon oxide", "miscellaneous/structures/SiO2.xyz"),
            ]),
    ],
    node_class='StructureData',
)
structure_selection_step = StructureSelectionStep(manager=structure_manager_widget)
structure_selection_step.auto_next = True
compute_bands_step = ComputeBandsSubmitWidget()

# Link the application steps
ipw.dlink((structure_selection_step, 'confirmed_structure'), (compute_bands_step, 'input_structure'))


# Add the application steps to the application
app = WizardApp(
    steps=[
        ('Select structure', structure_selection_step),
        ('Compute bands', compute_bands_step)])
compute_bands_step.callbacks.append(lambda _: app._update_titles())

# Add process selection header
work_chain_selector = WorkChainSelector(layout=ipw.Layout(width='auto'))

def _observe_process_selection(change):
    pk = change['new']
    if pk is None:
        compute_bands_step.process = None
    else:
        process = load_node(pk)
        with structure_manager_widget.hold_sync():
            with structure_selection_step.hold_sync():
                structure_manager_widget.structure = process.inputs.structure
                structure_selection_step.structure = process.inputs.structure
                structure_selection_step.confirmed_structure = process.inputs.structure
                compute_bands_step.process = process
        
work_chain_selector.observe(_observe_process_selection, 'value')    
ipw.dlink((compute_bands_step, 'process'), (work_chain_selector, 'value'),
          transform=lambda node: None if node is None else node.pk)

app_with_work_chain_selector = ipw.VBox(children=[work_chain_selector, app])

display(app_with_work_chain_selector)